<a href="https://colab.research.google.com/github/Albert-26/python-fundamentals/blob/main/Predictive_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Task 2, Question 1 Code Here
import pandas as pd
train_df = pd.read_csv("/content/drive/MyDrive/Dataset/train.csv")
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 61 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   ID                                            7000 non-null   int64  
 1   source                                        7000 non-null   object 
 2   name                                          6999 non-null   object 
 3   description                                   6943 non-null   object 
 4   neighborhood_overview                         5177 non-null   object 
 5   host_name                                     7000 non-null   object 
 6   host_since                                    7000 non-null   object 
 7   host_location                                 5720 non-null   object 
 8   host_about                                    4730 non-null   object 
 9   host_response_time                            7000 non-null   o

In [2]:
test_df = pd.read_csv("/content/drive/MyDrive/Dataset/test.csv")
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 60 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   ID                                            3000 non-null   int64  
 1   source                                        3000 non-null   object 
 2   name                                          3000 non-null   object 
 3   description                                   2969 non-null   object 
 4   neighborhood_overview                         1576 non-null   object 
 5   host_name                                     3000 non-null   object 
 6   host_since                                    3000 non-null   object 
 7   host_location                                 2230 non-null   object 
 8   host_about                                    1559 non-null   object 
 9   host_response_time                            2263 non-null   o

In [3]:
import numpy as np
column_name = train_df.columns
print('Checking the Number of NaN values per column in train dataset:')
for col in column_name:
    if int(train_df[col].isnull().sum()) > 0:  
      print(f'{col} has {train_df[col].isnull().sum()} missing points which is {np.round((train_df[col].isnull().sum()/7000)*100, 2)}% as missing value')

Checking the Number of NaN values per column in train dataset:
name has 1 missing points which is 0.01% as missing value
description has 57 missing points which is 0.81% as missing value
neighborhood_overview has 1823 missing points which is 26.04% as missing value
host_location has 1280 missing points which is 18.29% as missing value
host_about has 2270 missing points which is 32.43% as missing value
host_acceptance_rate has 63 missing points which is 0.9% as missing value
host_neighbourhood has 3460 missing points which is 49.43% as missing value
neighbourhood has 1823 missing points which is 26.04% as missing value
neighbourhood_cleansed has 108 missing points which is 1.54% as missing value
property_type has 84 missing points which is 1.2% as missing value
room_type has 134 missing points which is 1.91% as missing value
bathrooms has 6 missing points which is 0.09% as missing value
bedrooms has 378 missing points which is 5.4% as missing value
beds has 72 missing points which is 1.

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import re
import nltk
from tqdm import tqdm
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.sparse import hstack
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import absolute
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
import missingno as msno
from nltk.corpus import stopwords

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [5]:
import pandas as pd
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

# Install xgboost
!pip install xgboost

# Rest of the code
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV

# Continue with the rest of your code


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
train_df = pd.concat([train_df, test_df]) #merging test and train data for imputing all the missing values

In [7]:
#removing $ sign from the price column
train_df['price'] = train_df['price'].apply(lambda x: x.replace('$', '').replace(',', '') if isinstance(x, str) else x).astype(float)

# removing % sign from the host_response_rate column
train_df['host_response_rate'] = train_df['host_response_rate'].apply(lambda x: x.replace('%', '').replace(',', '') if isinstance(x, str) else x).astype(float)

# removing % sign from the host_response_rate column
train_df['host_acceptance_rate'] = train_df['host_acceptance_rate'].apply(lambda x: x.replace('%', '').replace(',', '')  if isinstance(x, str) else x).astype(float)


#extracting numberical values only from the bathroom column
train_df['bathrooms'] = train_df['bathrooms'].str.extract('(\d+)')
# imputing 0.5 in NaN as we have 'half shared bathrooms' which is not extracted from the column 
train_df['bathrooms'] = train_df['bathrooms'].fillna(0.5)

In [8]:
test_df = train_df[7000::]
train_df = train_df[:7000]

In [9]:
import pandas as pd
import datetime

# Function for feature engineering
def feature_engineering(data):
    data['name_length'] = data['name'].fillna('').apply(lambda x: len(x.split()))                                  #word count of the name
    data['desc_length'] = data['description'].fillna('').apply(lambda x: len(x.split()))                           #word count of the description
    data['neighborhood_overview_length'] = data['neighborhood_overview'].fillna('').apply(lambda x: len(x.split()))#word count of the neighbourhood overview
    data['host_about_length'] = data['host_about'].fillna('').apply(lambda x: len(x.split()))                       #word count of the host about
    data['total_verification'] = data['host_verifications'].fillna('').apply(lambda x: len(x.split()))              #number of verification of host
    data['total_amenities'] = data['amenities'].fillna('').apply(lambda x: len(x.split()))                          #total number of amenties provided by the host
    data['host_since'] = pd.to_datetime(data['host_since'])                                                          #converting into daytime format
    data['first_review'] = pd.to_datetime(data['first_review'])                                                      #converting into daytime format
    data['last_review'] = pd.to_datetime(data['last_review'])                                                        #converting into daytime format
    today = datetime.datetime.today()
    data['number_of_days_as_host'] = (today - data['host_since']).dt.days                                           #total number of days as host till today
    data['number_of_days_last_review'] = (today - data['last_review']).dt.days                                      #total number of day from last review till today
    data['number_of_days_first_review'] = (today - data['first_review']).dt.days                                    #total number of day from first review till today
    data['diff_in_last_&_first_review'] = data['number_of_days_first_review'] - data['number_of_days_last_review']  #days difference between first and last review
    data = data.drop(['host_since', 'first_review', 'last_review'], axis=1)
    data['neighbourhood'] = data['neighbourhood'].fillna('').apply(lambda x: (x.split(',')[0]))
    data['host_location'] = data['host_location'].fillna('').apply(lambda x: (x.split(',')[0]))
    return data

# Call the feature_engineering function for train and test datasets
train_df = feature_engineering(train_df)
test_df = feature_engineering(test_df)


<ipython-input-9-2d53ec538108>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['name_length'] = data['name'].fillna('').apply(lambda x: len(x.split()))                                  #word count of the name


In [10]:
train_df['name'] = train_df['name'].fillna('missing')
train_df['description'] = train_df['description'].fillna('missing')
train_df['neighborhood_overview'] = train_df['neighborhood_overview'].fillna('missing')
train_df['host_name'] = train_df['host_name'].fillna('missing')
train_df['host_location'] = train_df['host_location'].fillna(train_df['host_location'].mode()[0])
train_df['host_about'] = train_df['host_about'].fillna('missing')
train_df['host_response_time'] = train_df['host_response_time'].fillna(train_df['host_response_time'].mode()[0])
train_df['host_response_rate'] = train_df['host_response_rate'].fillna(train_df['host_response_rate'].mean())
train_df['host_acceptance_rate'] = train_df['host_acceptance_rate'].fillna(train_df['host_acceptance_rate'].mean())
train_df['host_is_superhost'] = train_df['host_is_superhost'].fillna(train_df['host_is_superhost'].mode()[0])
train_df['host_neighbourhood'] = train_df['host_neighbourhood'].fillna(train_df['host_neighbourhood'].mode()[0])
train_df['host_listings_count'] = train_df['host_listings_count'].fillna(train_df['host_listings_count'].mean())
train_df['host_verifications'] = train_df['host_verifications'].fillna(train_df['host_verifications'].mode()[0])
train_df['host_has_profile_pic'] = train_df['host_has_profile_pic'].fillna(train_df['host_has_profile_pic'].mode()[0])
train_df['host_identity_verified'] = train_df['host_identity_verified'].fillna(train_df['host_identity_verified'].mode()[0])
train_df['neighbourhood'] = train_df['neighbourhood'].fillna(train_df['neighbourhood'].mode()[0])
train_df['neighbourhood_cleansed'] = train_df['neighbourhood_cleansed'].fillna(train_df['neighbourhood_cleansed'].mode()[0])
train_df['latitude'] = train_df['latitude'].fillna(train_df['latitude'].mean())
train_df['longitude'] = train_df['longitude'].fillna(train_df['longitude'].mean())
train_df['property_type'] = train_df['property_type'].fillna(train_df['property_type'].mode()[0])
train_df['room_type'] = train_df['room_type'].fillna(train_df['room_type'].mode()[0])
train_df['accommodates'] = train_df['accommodates'].fillna(train_df['accommodates'].mean())
train_df['bathrooms'] = pd.to_numeric(train_df['bathrooms'], errors='coerce')
train_df['bathrooms'] = train_df['bathrooms'].fillna(train_df['bathrooms'].mean())
train_df['bedrooms'] = train_df['bedrooms'].fillna(train_df['bedrooms'].mean())
train_df['beds'] = train_df['beds'].fillna(train_df['beds'].mean())
train_df['amenities'] = train_df['amenities'].fillna('missing')
train_df['minimum_nights'] = train_df['minimum_nights'].fillna(train_df['minimum_nights'].mean())
train_df['maximum_nights'] = train_df['maximum_nights'].fillna(train_df['maximum_nights'].mean())
train_df['minimum_minimum_nights'] = train_df['minimum_minimum_nights'].fillna(train_df['minimum_minimum_nights'].mean())
train_df['maximum_minimum_nights'] = train_df['maximum_minimum_nights'].fillna(train_df['maximum_minimum_nights'].mean())
train_df['minimum_maximum_nights'] = train_df['minimum_maximum_nights'].fillna(train_df['minimum_maximum_nights'].mean())
train_df['maximum_maximum_nights'] = train_df['maximum_maximum_nights'].fillna(train_df['maximum_maximum_nights'].mean())
train_df['minimum_nights_avg_ntm'] = train_df['minimum_nights_avg_ntm'].fillna(train_df['minimum_nights_avg_ntm'].mean())
train_df['maximum_nights_avg_ntm'] = train_df['maximum_nights_avg_ntm'].fillna(train_df['maximum_nights_avg_ntm'].mean())
train_df['has_availability'] = train_df['has_availability'].fillna(train_df['has_availability'].mode()[0])
train_df['availability_30'] = train_df['availability_30'].fillna(train_df['availability_30'].mean())
train_df['availability_60'] = train_df['availability_60'].fillna(train_df['availability_60'].mean())
train_df['availability_90'] = train_df['availability_90'].fillna(train_df['availability_90'].mean())
train_df['availability_365'] = train_df['availability_365'].fillna(train_df['availability_365'].mean())
train_df['number_of_reviews'] = train_df['number_of_reviews'].fillna(train_df['number_of_reviews'].mean())
train_df['number_of_reviews_ltm'] = train_df['number_of_reviews_ltm'].fillna(train_df['number_of_reviews_ltm'].mean())
train_df['number_of_reviews_l30d'] = train_df['number_of_reviews_l30d'].fillna(train_df['number_of_reviews_l30d'].mean())
train_df['review_scores_rating'] = train_df['review_scores_rating'].fillna(train_df['review_scores_rating'].mean())
train_df['review_scores_accuracy'] = train_df['review_scores_accuracy'].fillna(train_df['review_scores_accuracy'].mean())
train_df['review_scores_cleanliness'] = train_df['review_scores_cleanliness'].fillna(train_df['review_scores_cleanliness'].mean())
train_df['review_scores_checkin'] = train_df['review_scores_checkin'].fillna(train_df['review_scores_checkin'].mean())
train_df['review_scores_communication'] = train_df['review_scores_communication'].fillna(train_df['review_scores_communication'].mean())
train_df['review_scores_location'] = train_df['review_scores_location'].fillna(train_df['review_scores_location'].mean())
train_df['review_scores_value'] = train_df['review_scores_value'].fillna(train_df['review_scores_accuracy'].mean())
train_df['instant_bookable'] = train_df['instant_bookable'].fillna(train_df['instant_bookable'].mode()[0])
train_df['calculated_host_listings_count'] = train_df['calculated_host_listings_count'].fillna(train_df['calculated_host_listings_count'].mean())
train_df['calculated_host_listings_count_entire_homes'] = train_df['calculated_host_listings_count_entire_homes'].fillna(train_df['calculated_host_listings_count_entire_homes'].mean())
train_df['calculated_host_listings_count_private_rooms'] = train_df['calculated_host_listings_count_private_rooms'].fillna(train_df['calculated_host_listings_count_private_rooms'].mean())
train_df['calculated_host_listings_count_shared_rooms'] = train_df['calculated_host_listings_count_shared_rooms'].fillna(train_df['calculated_host_listings_count_shared_rooms'].mean())
train_df['reviews_per_month'] = train_df['reviews_per_month'].fillna(train_df['reviews_per_month'].mean())
train_df['reviews_per_month'] = train_df['reviews_per_month'].fillna(train_df['reviews_per_month'].mean())
train_df['price'] = train_df['price'].fillna(train_df['price'].mean())
train_df['name_length'] = train_df['name_length'].fillna(train_df['name_length'].mean())
train_df['desc_length'] = train_df['desc_length'].fillna(train_df['desc_length'].mean())
train_df['neighborhood_overview_length'] = train_df['neighborhood_overview_length'].fillna(train_df['neighborhood_overview_length'].mean())
train_df['host_about_length'] = train_df['host_about_length'].fillna(train_df['host_about_length'].mean())
train_df['total_verification'] = train_df['total_verification'].fillna(train_df['total_verification'].mean())
train_df['total_amenities'] = train_df['total_amenities'].fillna(train_df['total_amenities'].mean())
train_df['number_of_days_as_host'] = train_df['number_of_days_as_host'].fillna(train_df['number_of_days_as_host'].mean())
train_df['number_of_days_last_review'] = train_df['number_of_days_last_review'].fillna(train_df['number_of_days_last_review'].mean())
train_df['number_of_days_first_review'] = train_df['number_of_days_first_review'].fillna(train_df['number_of_days_first_review'].mean())
train_df['diff_in_last_&_first_review'] = train_df['diff_in_last_&_first_review'].fillna(train_df['diff_in_last_&_first_review'].mean())


In [11]:
column_names = test_df.columns
print(column_names)

Index(['ID', 'source', 'name', 'description', 'neighborhood_overview',
       'host_name', 'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_neighbourhood', 'host_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'amenities', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
       'number_of_reviews_l30d', 'review_scores_rating',
       'review_scores_accuracy', 'review_sc

In [12]:
test_df['name'] = test_df['name'].fillna('missing')
test_df['description'] = test_df['description'].fillna('missing')
test_df['neighborhood_overview'] = test_df['neighborhood_overview'].fillna('missing')
test_df['host_name'] = test_df['host_name'].fillna('missing')
test_df['host_location'] = test_df['host_location'].fillna(test_df['host_location'].mode()[0])
test_df['host_about'] = test_df['host_about'].fillna('missing')
test_df['host_response_time'] = test_df['host_response_time'].fillna(test_df['host_response_time'].mode()[0])
test_df['host_response_rate'] = test_df['host_response_rate'].fillna(test_df['host_response_rate'].mean())
test_df['host_acceptance_rate'] = test_df['host_acceptance_rate'].fillna(test_df['host_acceptance_rate'].mean())
test_df['host_is_superhost'] = test_df['host_is_superhost'].fillna(test_df['host_is_superhost'].mode()[0])
test_df['host_neighbourhood'] = test_df['host_neighbourhood'].fillna(test_df['host_neighbourhood'].mode()[0])
test_df['host_listings_count'] = test_df['host_listings_count'].fillna(test_df['host_listings_count'].mean())
test_df['host_verifications'] = test_df['host_verifications'].fillna(test_df['host_verifications'].mode()[0])
test_df['host_has_profile_pic'] = test_df['host_has_profile_pic'].fillna(test_df['host_has_profile_pic'].mode()[0])
test_df['host_identity_verified'] = test_df['host_identity_verified'].fillna(test_df['host_identity_verified'].mode()[0])
test_df['neighbourhood'] = test_df['neighbourhood'].fillna(test_df['neighbourhood'].mode()[0])
test_df['neighbourhood_cleansed'] = test_df['neighbourhood_cleansed'].fillna(test_df['neighbourhood_cleansed'].mode()[0])
test_df['latitude'] = test_df['latitude'].fillna(test_df['latitude'].mean())
test_df['longitude'] = test_df['longitude'].fillna(test_df['longitude'].mean())
test_df['property_type'] = test_df['property_type'].fillna(test_df['property_type'].mode()[0])
test_df['room_type'] = test_df['room_type'].fillna(test_df['room_type'].mode()[0])
test_df['accommodates'] = test_df['accommodates'].fillna(test_df['accommodates'].mean())
test_df['bathrooms'] = pd.to_numeric(test_df['bathrooms'], errors='coerce')
test_df['bathrooms'] = test_df['bathrooms'].fillna(test_df['bathrooms'].mean())
test_df['bedrooms'] = test_df['bedrooms'].fillna(test_df['bedrooms'].mean())
test_df['beds'] = test_df['beds'].fillna(test_df['beds'].mean())
test_df['amenities'] = test_df['amenities'].fillna('missing')
test_df['minimum_nights'] = test_df['minimum_nights'].fillna(test_df['minimum_nights'].mean())
test_df['maximum_nights'] = test_df['maximum_nights'].fillna(test_df['maximum_nights'].mean())
test_df['minimum_minimum_nights'] = test_df['minimum_minimum_nights'].fillna(test_df['minimum_minimum_nights'].mean())
test_df['maximum_minimum_nights'] = test_df['maximum_minimum_nights'].fillna(test_df['maximum_minimum_nights'].mean())
test_df['minimum_maximum_nights'] = test_df['minimum_maximum_nights'].fillna(test_df['minimum_maximum_nights'].mean())
test_df['maximum_maximum_nights'] = test_df['maximum_maximum_nights'].fillna(test_df['maximum_maximum_nights'].mean())
test_df['minimum_nights_avg_ntm'] = test_df['minimum_nights_avg_ntm'].fillna(test_df['minimum_nights_avg_ntm'].mean())
test_df['maximum_nights_avg_ntm'] = test_df['maximum_nights_avg_ntm'].fillna(test_df['maximum_nights_avg_ntm'].mean())
test_df['has_availability'] = test_df['has_availability'].fillna(test_df['has_availability'].mode()[0])
test_df['availability_30'] = test_df['availability_30'].fillna(test_df['availability_30'].mean())
test_df['availability_60'] = test_df['availability_60'].fillna(test_df['availability_60'].mean())
test_df['availability_90'] = test_df['availability_90'].fillna(test_df['availability_90'].mean())
test_df['availability_365'] = test_df['availability_365'].fillna(test_df['availability_365'].mean())
test_df['number_of_reviews'] = test_df['number_of_reviews'].fillna(test_df['number_of_reviews'].mean())
test_df['number_of_reviews_ltm'] = test_df['number_of_reviews_ltm'].fillna(test_df['number_of_reviews_ltm'].mean())
test_df['number_of_reviews_l30d'] = test_df['number_of_reviews_l30d'].fillna(test_df['number_of_reviews_l30d'].mean())
test_df['review_scores_rating'] = test_df['review_scores_rating'].fillna(test_df['review_scores_rating'].mean())
test_df['review_scores_accuracy'] = test_df['review_scores_accuracy'].fillna(test_df['review_scores_accuracy'].mean())
test_df['review_scores_cleanliness'] = test_df['review_scores_cleanliness'].fillna(test_df['review_scores_cleanliness'].mean())
test_df['review_scores_checkin'] = test_df['review_scores_checkin'].fillna(test_df['review_scores_checkin'].mean())
test_df['review_scores_communication'] = test_df['review_scores_communication'].fillna(test_df['review_scores_communication'].mean())
test_df['review_scores_location'] = test_df['review_scores_location'].fillna(test_df['review_scores_location'].mean())
test_df['review_scores_value'] = test_df['review_scores_value'].fillna(test_df['review_scores_accuracy'].mean())
test_df['instant_bookable'] = test_df['instant_bookable'].fillna(test_df['instant_bookable'].mode()[0])
test_df['calculated_host_listings_count'] = test_df['calculated_host_listings_count'].fillna(test_df['calculated_host_listings_count'].mean())
test_df['calculated_host_listings_count_entire_homes'] = test_df['calculated_host_listings_count_entire_homes'].fillna(test_df['calculated_host_listings_count_entire_homes'].mean())
test_df['calculated_host_listings_count_private_rooms'] = test_df['calculated_host_listings_count_private_rooms'].fillna(test_df['calculated_host_listings_count_private_rooms'].mean())
test_df['calculated_host_listings_count_shared_rooms'] = test_df['calculated_host_listings_count_shared_rooms'].fillna(test_df['calculated_host_listings_count_shared_rooms'].mean())
test_df['reviews_per_month'] = test_df['reviews_per_month'].fillna(test_df['reviews_per_month'].mean())
test_df['reviews_per_month'] = test_df['reviews_per_month'].fillna(test_df['reviews_per_month'].mean())
test_df['price'] = test_df['price'].fillna(test_df['price'].mean())
test_df['name_length'] = test_df['name_length'].fillna(test_df['name_length'].mean())
test_df['desc_length'] = test_df['desc_length'].fillna(test_df['desc_length'].mean())
test_df['neighborhood_overview_length'] = test_df['neighborhood_overview_length'].fillna(test_df['neighborhood_overview_length'].mean())
test_df['host_about_length'] = test_df['host_about_length'].fillna(test_df['host_about_length'].mean())
test_df['total_verification'] = test_df['total_verification'].fillna(test_df['total_verification'].mean())
test_df['total_amenities'] = test_df['total_amenities'].fillna(test_df['total_amenities'].mean())
test_df['number_of_days_as_host'] = test_df['number_of_days_as_host'].fillna(test_df['number_of_days_as_host'].mean())
test_df['number_of_days_last_review'] = test_df['number_of_days_last_review'].fillna(test_df['number_of_days_last_review'].mean())
test_df['number_of_days_first_review'] = test_df['number_of_days_first_review'].fillna(test_df['number_of_days_first_review'].mean())
test_df['diff_in_last_&_first_review'] = test_df['diff_in_last_&_first_review'].fillna(test_df['diff_in_last_&_first_review'].mean())

In [13]:
# Task 2, Question 4 Code Here
# replacing 't' with 0 and 'f' with 1 in all of the below column for encoding categorical variable
train_df['host_is_superhost'] = train_df['host_is_superhost'].apply(lambda x: 0 if x=='t' else 1)
train_df['host_has_profile_pic'] = train_df['host_has_profile_pic'].apply(lambda x: 0 if x=='t' else 1)
train_df['host_identity_verified'] = train_df['host_identity_verified'].apply(lambda x: 0 if x=='t' else 1)
train_df['has_availability'] = train_df['has_availability'].apply(lambda x: 0 if x=='t' else 1)
train_df['instant_bookable'] = train_df['instant_bookable'].apply(lambda x: 0 if x=='t' else 1)

In [14]:
# replacing 't' with 0 and 'f' with 1 in all of the below column for encoding categorical variable
test_df['host_is_superhost'] = test_df['host_is_superhost'].apply(lambda x: 0 if x=='t' else 1)
test_df['host_has_profile_pic'] = test_df['host_has_profile_pic'].apply(lambda x: 0 if x=='t' else 1)
test_df['host_identity_verified'] = test_df['host_identity_verified'].apply(lambda x: 0 if x=='t' else 1)
test_df['has_availability'] = test_df['has_availability'].apply(lambda x: 0 if x=='t' else 1)
test_df['instant_bookable'] = test_df['instant_bookable'].apply(lambda x: 0 if x=='t' else 1)

In [15]:
#One hot encoding for the categorical feature source
vectorizer = CountVectorizer()
train_source_ohe = vectorizer.fit_transform(train_df['source'].values)
test_source_ohe = vectorizer.transform(test_df['source'].values)

In [16]:
#One hot encoding for the categorical feature host_name
vectorizer = CountVectorizer()
train_host_name_ohe = vectorizer.fit_transform(train_df['host_name'].values)
test_host_name_ohe = vectorizer.transform(test_df['host_name'].values)

In [17]:
#One hot encoding for the categorical feature  host location
vectorizer = CountVectorizer()
train_host_location_ohe = vectorizer.fit_transform(train_df['host_location'].values)
test_host_location_ohe = vectorizer.transform(test_df['host_location'].values)

In [18]:
# Fill NaN values in host_neighbourhood column
train_df['host_neighbourhood'] = train_df['host_neighbourhood'].fillna('missing')
test_df['host_neighbourhood'] = test_df['host_neighbourhood'].fillna('missing')

# Apply CountVectorizer to host_neighbourhood column
vectorizer = CountVectorizer()
train_host_neighbourhood_ohe = vectorizer.fit_transform(train_df['host_neighbourhood'].values)
test_host_neighbourhood_ohe = vectorizer.transform(test_df['host_neighbourhood'].values)

In [19]:
#One hot encoding for the categorical feature host verifications
vectorizer = CountVectorizer()
train_host_verifications_ohe = vectorizer.fit_transform(train_df['host_verifications'].values)
test_host_verifications_ohe = vectorizer.transform(test_df['host_verifications'].values)

In [20]:
#One hot encoding for the categorical feature neighbourhood
vectorizer = CountVectorizer()
train_neighbourhood_ohe = vectorizer.fit_transform(train_df['neighbourhood'].values)
test_neighbourhood_ohe = vectorizer.transform(test_df['neighbourhood'].values)

In [21]:
# Fill NaN values in host_response_time column
train_df['host_response_time'] = train_df['host_response_time'].fillna('missing')
test_df['host_response_time'] = test_df['host_response_time'].fillna('missing')

# Apply CountVectorizer to host_response_time column
vectorizer = CountVectorizer()
train_host_response_time_ohe = vectorizer.fit_transform(train_df['host_response_time'].values)
test_host_response_time_ohe = vectorizer.transform(test_df['host_response_time'].values)

In [22]:
# Fill NaN values in neighbourhood_cleansed column
train_df['neighbourhood_cleansed'] = train_df['neighbourhood_cleansed'].fillna('missing')
test_df['neighbourhood_cleansed'] = test_df['neighbourhood_cleansed'].fillna('missing')

# Apply CountVectorizer to neighbourhood_cleansed column
vectorizer = CountVectorizer()
train_neighbourhood_cleansed_ohe = vectorizer.fit_transform(train_df['neighbourhood_cleansed'].values)
test_neighbourhood_cleansed_ohe = vectorizer.transform(test_df['neighbourhood_cleansed'].values)


In [23]:
#One hot encoding for the categorical feature amenties
vectorizer = CountVectorizer()
train_amenities_ohe = vectorizer.fit_transform(train_df['amenities'].values)
test_amenities_ohe = vectorizer.transform(test_df['amenities'].values)

In [24]:
#cleaning text feature by removing stopwords and all the other character like html, puntuation etc

# https://stackoverflow.com/a/47091490/4084039
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'

stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

def preprocess_text(text_data):
    preprocessed_text = []
    # tqdm is for printing the status bar
    for sentance in tqdm(text_data):
        sent = decontracted(sentance)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\n', ' ')
        sent = sent.replace('\\"', ' ')
        sent = sent.replace('()', '')
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        # https://gist.github.com/sebleier/554280
        sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
        preprocessed_text.append(sent.lower().strip())
    return preprocessed_text

In [25]:
import re
import nltk
from tqdm import tqdm

def preprocess_text(text_data):
    # Function to clean and preprocess text data
    cleaned_data = []

    # tqdm is for printing the status bar
    for sentence in tqdm(text_data):
        if isinstance(sentence, str):  # Check if the sentence is a string
            sent = decontracted(sentence)
            sent = sent.replace('\\r', ' ')
            sent = sent.replace('\\n', ' ')
            sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
            sent = ' '.join(e.lower() for e in sent.split() if e.lower() not in stopwords.words('english'))
            cleaned_data.append(sent)
        else:
            cleaned_data.append('')  # Fill non-string values with an empty string

    return cleaned_data


In [26]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
train_df['clean_name'] = preprocess_text(train_df['name']) #cleaning name feature for train dataset
train_df['clean_desc'] = preprocess_text(train_df['description']) #cleaning description feature for train dataset
train_df['clean_neighborhood_overview'] = preprocess_text(train_df['neighborhood_overview'])  #cleaning neighbourhood overview feature for train dataset

test_df['clean_name'] = preprocess_text(test_df['name'])  #cleaning name feature for test dataset
test_df['clean_desc'] = preprocess_text(test_df['description']) #cleaning description feature for test dataset
test_df['clean_neighborhood_overview'] = preprocess_text(test_df['neighborhood_overview'])  #cleaning neighbourhood overview feature for test dataset

100%|██████████| 3000/3000 [00:12<00:00, 249.39it/s]


In [28]:
# implementing tfidf word vectorizer into name
vectorizer = TfidfVectorizer(min_df=10)
train_name_tfidf = vectorizer.fit_transform(train_df['clean_name'].values)
test_name_tfidf = vectorizer.transform(test_df['clean_name'].values)

In [29]:
# implementing tfidf word vectorizer into description
vectorizer = TfidfVectorizer(min_df=10)
train_desc_tfidf = vectorizer.fit_transform(train_df['clean_desc'].values)
test_desc_tfidf = vectorizer.transform(test_df['clean_desc'].values)

In [30]:
#Implementing NLTK sentimental intensity analyser on the clean preprocessed text feature
def SID(text):  
    ''' This function calculates the NLTK sentiments and return the negative, neutral, postive and compound values'''
    negative = []
    neutral = []
    positive = []
    compound = []
    for lines in tqdm(text):
      sid = SentimentIntensityAnalyzer()
      sentiment_score = sid.polarity_scores(lines)
    
      negative.append(sentiment_score['neg'])
      neutral.append(sentiment_score['neu'])
      positive.append(sentiment_score['pos'])
      compound.append(sentiment_score['compound'])
    negative = np.asarray(negative).reshape(-1,1)
    neutral = np.asarray(neutral).reshape(-1,1)
    positive = np.asarray(positive).reshape(-1,1)
    compound = np.asarray(compound).reshape(-1,1)
    return negative, neutral, positive, compound

train_name_neg, train_name_neu, train_name_pos, train_name_comp = SID(train_df['clean_name'].values) #for name feature
train_desc_neg, train_desc_neu, train_desc_pos, train_desc_comp = SID(train_df['clean_desc'].values)  #for description feature
train_neigh_over_neg, train_neigh_over_neu, train_neigh_over_pos, train_neigh_over_comp = SID(train_df['clean_neighborhood_overview'].values) #for neighbourhood overview feature

test_name_neg, test_name_neu, test_name_pos, test_name_comp = SID(test_df['clean_name'].values) #for name feature
test_desc_neg, test_desc_neu, test_desc_pos, test_desc_comp = SID(test_df['clean_desc'].values) #for name feature
test_neigh_over_neg, test_neigh_over_neu, test_neigh_over_pos, test_neigh_over_comp = SID(test_df['clean_neighborhood_overview'].values) #for name feature

100%|██████████| 3000/3000 [00:26<00:00, 111.55it/s]


In [31]:
## Task 2, Question 5 Code Here
## scaling all the numerical data feature
scale_list = ['name_length', 'desc_length', 'neighborhood_overview_length','bathrooms','host_about_length', 'total_verification', 'total_amenities','number_of_days_as_host','number_of_days_last_review' , 'number_of_days_first_review', 'diff_in_last_&_first_review','host_response_rate', 'host_acceptance_rate', 'latitude', 'longitude', 'maximum_nights','maximum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_nights_avg_ntm', 'availability_90', 'number_of_reviews', 'review_scores_rating','calculated_host_listings_count_private_rooms','calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_shared_rooms', 'reviews_per_month', 'calculated_host_listings_count', 'review_scores_value', 'review_scores_location', 'review_scores_communication', 'review_scores_checkin', 'review_scores_cleanliness', 'review_scores_accuracy', 'review_scores_rating', 'number_of_reviews_l30d', 'number_of_reviews_ltm', 'availability_365', 'availability_90', 'availability_60', 'availability_30', 'maximum_nights_avg_ntm', 'minimum_nights_avg_ntm', 'maximum_minimum_nights', 'minimum_minimum_nights', 'maximum_nights', 'minimum_nights', 'host_listings_count']
scale_list = list(set(scale_list))
for col in scale_list:
    scal = MinMaxScaler()
    train_df[col] = scal.fit_transform(train_df[col].values.reshape(-1,1))
    test_df[col] = scal.transform(test_df[col].values.reshape(-1,1))

In [32]:
#Task 3 code here
#droping all the dataset
drop_columns_list = ['source', 'host_name', 'host_location', 'price', 'host_about', 'host_neighbourhood', 'name', 'description', 'neighborhood_overview', 'clean_name', 'clean_desc', 'clean_neighborhood_overview', 'neighbourhood','host_response_time', 'neighbourhood_cleansed', 'host_verifications', 'property_type', 'room_type', 'amenities']

Y = train_df['price']
train_df = train_df.drop(drop_columns_list, axis=1)
test_df = test_df.drop(drop_columns_list, axis=1)


In [33]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# Assuming your data is in pandas dataframes train_df and test_df
X_train = train_df

# Assuming target variable price is in variable Y
y_train = Y

X_test = test_df   # As you mentioned, test data doesn't have a 'price' column

# Defining the model
lr = LinearRegression()

# Defining parameter grids for grid search
parameters_lr = {'fit_intercept': [True, False]}

# Grid search for hyperparameter tuning
print('Starting training for Linear Regression')
grid_search = GridSearchCV(estimator=lr, param_grid=parameters_lr, cv=5)
grid_search.fit(X_train, y_train)

# Print best parameters
print(f'Best parameters for Linear Regression: {grid_search.best_params_}')

# Refit the model with best parameters
lr = grid_search.best_estimator_

# Predict the target for training data
y_train_pred = lr.predict(X_train)

# Accuracy scores
rmse = sqrt(mean_squared_error(y_train, y_train_pred))
r2 = r2_score(y_train, y_train_pred)

print(f'Scores for Linear Regression:')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

# Predict on the test dataset
y_test_pred = lr.predict(X_test)

# Create a dataframe for output
output_df = pd.DataFrame({
    'ID': test_df['ID'],   # assuming the ID column in your test dataset is named 'ID'
    'price': y_test_pred
})

# Save to a csv file
output_df.to_csv('predictions.csv', index=False)


Starting training for Linear Regression
Best parameters for Linear Regression: {'fit_intercept': True}
Scores for Linear Regression:
RMSE: 2315.766251463929
R^2: 0.008172298488895224


In [34]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# Assuming your data is in pandas dataframes train_df and test_df
X_train = train_df
y_train = Y
X_test = test_df   # As you mentioned, test data doesn't have a 'price' column

# Defining the models
models = {
    'Linear Regression': LinearRegression(),
    'SVR': SVR(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'XGBRegressor': XGBRegressor(objective ='reg:squarederror')
}

# Defining parameter grids for grid search
parameters = {
    'Linear Regression': {'fit_intercept': [True, False]},
    'SVR': {'kernel': ['linear', 'rbf'], 'C': [1, 10]},
    'Random Forest': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]},
    'Gradient Boosting': {'n_estimators': [100, 200], 'learning_rate': [0.1, 0.01, 0.001], 'max_depth': [3, 5, 7]},
    'XGBRegressor': {'n_estimators': [100, 200], 'learning_rate': [0.1, 0.01, 0.001], 'max_depth': [3, 5, 7]}
}


In [ ]:
best_models = {}
for model_name, model in models.items():
    print(f'Starting training for {model_name}')
    grid_search = GridSearchCV(estimator=model, param_grid=parameters[model_name], cv=5)
    grid_search.fit(X_train, y_train)
    
    # Print best parameters
    print(f'Best parameters for {model_name}: {grid_search.best_params_}')

    # Refit the model with best parameters
    model = grid_search.best_estimator_

    # Predict the target for training data
    y_train_pred = model.predict(X_train)
    
    # Accuracy scores
    rmse = sqrt(mean_squared_error(y_train, y_train_pred))
    r2 = r2_score(y_train, y_train_pred)
    
    print(f'Scores for {model_name}:')
    print(f'RMSE: {rmse}')
    print(f'R^2: {r2}')
    
    best_models[model_name] = model


Starting training for Linear Regression
Best parameters for Linear Regression: {'fit_intercept': True}
Scores for Linear Regression:
RMSE: 2315.766251463929
R^2: 0.008172298488895224
Starting training for SVR
